Welcome to lesson 6 of the Noisebridge Python class! ([Noisebridge Wiki](https://www.noisebridge.net/wiki/PyClass) | [Github](https://github.com/audiodude/PythonClass))

In this lesson, we will learn about Object Oriented Programming and how classes and objects work in Python. We will also discuss decorators.

You will learn:

* How to define a class, and instantiate an object of that class
* What a constructor is in OOP
* Proper uses of the Python `self` variable
* How object properties are stored and calculated
* Basic inheritance/class hierarchies
* How to use and define decorators

Let's go!

An **object** is a data structure that encapsulates data (**instance variables**) and functions that operate on that data (**methods**). An object is created by **instantiating** a **class**, where the class can be thought of as the template for the object.

In [7]:
class Fruit:
  store = 'Safeway'
  
  def __init__(self, name, price):
    print('In Fruit constructor __init__')
    self.name = name
    self.price = price

  def display(self):
    print(f'{self.name} is ${self.price:.2f}')

  def discount(self, percent):
    self.price *= (1 - (percent / 100))
    if not hasattr(self, 'discount_percent'):
      self.discount_percent = percent
    else:
      self.discount_percent = self.discount_percent + (100 - self.discount_percent) * (1 - (percent/100))

  def double_discount(self, percent):
    self.discount(percent * 2)

apple = Fruit('apple', 1.49)
apple.display()
apple.discount(20)
apple.display()
apple.discount(50)
print(apple.discount_percent)
apple.in_stock = True

In Fruit constructor __init__
apple is $1.49
apple is $1.19
60.0


We've already seen a few of the features of classes and objects:

* Objects are created (or "instantiated") by "calling" their class objects (`Fruit()`)
* Objects can have properties (or "instance variables"), such as `a.name`
* In Python, the first argument to an instance method (a method on the class that operates on an object of that class) is always `self`, which can be used to access the other properties and methods of the object.
* Properties can be added to objects, and they don't need to be defined ahead of time.

Let's consider each of these points

## Constructors

In OOP (Object Oriented Programming), a constructor is a method that is used to construct, or instantiate, a member of the class (an object). So in the above example, `Fruit` is the *class*, `a` is the *object*, and it is constructed using the syntax `Fruit('apple', 1.49)`. When a class is called in this way and an object is constructed, the special `__init__` (double underscore init) method is called. This method is the constructor, which is similar in purpose to constructors in other OOP languages.

Constructors are where you "set up" your object. In this example, our objects need a name and a price. Luckily, both of those are passed in as arguments to the constructor, so we simply assign them using the syntax `self.name = name` (same for price).

Constructors can have any number of positional or keyword arguments, just like regular functions, though the first positional argument is always `self`.

In many cases, a constructor will be where the properties, or instance variables, of an object are initialized, such as we just did with `name` and `price`.

## `self`

Python passes the object itself as the first argument to every instance method. It is almost universally implemented to assign this to a parameter named `self`, though technically the name could be anything. The `self` parameter operates very similarly to `this` in other OOP languages. It allows you to access member variables and methods of the "current" object.

So the call a.discount(20) calls the `discount` method of the `Fruit` object, with `self == a` essentially. Also, in the `double_discount` method, we can use the `self` variable to call other methods of the object, like `discount`.

## Properties

We have already seen some examples of object properties in our Python code. It might help to understand properties if we refer to them by their other name: instance variables. Basically, they are variables that are "scoped" to a specific instance of an object. So we can have two different fruit objects, both with a `price` property, but the prices can be different. Changing one will not affect the other.

Also note that properties do not have to be "declared" or "defined" anywhere. This is different from other OOP languages like Java, where you have to state upfront "Objects of this class will have a property 'name' which is a String" or whatever. While we will often add properties to objects in their constructors, it is not strictly necessary. Objects don't need to have the same set of properties defined during their lifetime, and we have already seen an example of this. The `Fruit` objects we defined only have an `discount_percent` property if the `discount` method has been called on them. We can check for the existence of a property on an object using the built in `hasattr`:

In [ ]:
name = 'Pedro'
print(hasattr(name, 'startswith'))
print(hasattr(name, 'party'))
# This kind of works like the dict `get` method.
print(getattr(name, '__class__'))
print(getattr(name, '__name__', '--does not exist--'))



In fact, in Python, you can add any properties you like to any object without raising an error. Though doing so will likely make your code hard to read and debug. It is especially error prone to overwrite attributes, like `name`, `price` and `discount_percent` that are used by the class methods, since those methods might make certain assumptions about the values of those attributes. In this case, for example, the Fruit class assumes that `discount_percent` is a running total of how much has been discounted from the price, and uses it to calculate its next value.

In [ ]:
a_fruit = Fruit('generic', 5.00)

a_fruit.why = 1
a_fruit.would = 2
a_fruit.you = 3
a_fruit.do = 4
a_fruit.this = 5

print(a_fruit.you)

In Python, we can use `hasattr` to check if a property exists on an object, as we did above. We can also use `getattr` and `setattr` in the case that we want to get or set a property, but the name is dynamic (not known when we are writing the code).

In [ ]:
import time

assert hasattr(a_fruit, 'name')
assert getattr(a_fruit, 'price')

# This has no practical purpose, except to demonstrate that the
# property name is effectively random and we don't know it ahead
# of time.
prop = 'quantity%.0f' % time.time()
print(prop)

setattr(a_fruit, prop, 10)
print(getattr(a_fruit, prop))

Classes can also feature instance variables that are initialized to the same value no matter the object, by placing these variables at the "top level" of the class definition (alongside the methods).

In [ ]:
class Tool:
  workbench = 'Home'

  def __init__(self, name, use):
    self.name = name
    self.use = use

  def get_workbench(self):
    return self.workbench

hammer = Tool('hammer', 'pounding')
screwdriver = Tool('screwdriver', 'screwing')

print(f'{hammer.name} workbench is {hammer.get_workbench()}')
print(f'{screwdriver.name} workbench is {screwdriver.get_workbench()}')

hammer.workbench = 'Garage'

print(f'{hammer.name} workbench is {hammer.get_workbench()}')
print(f'{screwdriver.name} workbench is {screwdriver.get_workbench()}')

## Inheritance

While it is very useful to encapsulate logic into a class/object, another useful feature of OOP is class **inheritance** or **class hierarchies**. Let's look at an example.

In [ ]:
class MultiFruit(Fruit):
    def __init__(self, name, price, quantity):
        print('In MultiFruit constructor __init__')
        super().__init__(name, price)
        print('Back in MultiFruit constructor')
        self.quantity = quantity
        
    def per_item_price(self):
        return round(self.price / self.quantity, 2)
    
grapes = MultiFruit('grapes', 0.89, 10)
print(grapes.per_item_price())
grapes.discount(10)
print('Grapes cost %s, or %s per grape' % (grapes.price, grapes.per_item_price()))

Here, we've created an "is a" relationship between `MultiFruit` and `Fruit`. Any object that is a MultiFruit, "is a" Fruit (but the opposite is not true!). The `MultiFruit` is said to **inherit** the definitions of `discount` and `double_discount` from it's **superclass** (the class that's directly "above" it in the class hierarchy, as if we were drawing a tree diagram). Because of this inheritance, we can call `grapes.discount()` on `grapes` just like on any Fruit (because it "is a" Fruit).

The `per_item_price()` method exists on `MultiFruit`, but not on Fruit. This reinforces the idea that a `MultiFruit` is a `Fruit` but a `Fruit` is not necessarily a `MultiFruit`. It wouldn't make sense for `Fruit` to have a `per_item_price()` method, since it doesn't have a quantity.

Also notice the use of `super()` in the constructor. Here, we want to initialize our `MultiFruit` with a name and price, just like we do for regular `Fruit`. In this simple example, we could have just initialized those properties directly and skipped the call to the superclass constructor:

In [ ]:
class MultiFruit2(Fruit):
    def __init__(self, name, price, quantity):
        print('In MultiFruit2 constructor __init__')
        self.name = name
        self.price = price
        self.quantity = quantity

However, it's usually a good practice to try to use the superclass constructor in **subclasses**, since we might later update the `Fruit` constructor to, say, calculate ripeness on initialization, and it could introduce bugs (fruit flies??) if `MultiFruit` was being initialized in a totally separate way.

When we define a subclass, we can **inherit** methods or **override** them. In our `MultiFruit2` example, we override the constructor in the fruit class. When we instantiate a `MultiFruit2`, the constructor in the `MultiFruit2` class definition is called *instead* of the one in the `Fruit` class definition. However, when we call `display()`, we inherit that method from the `Fruit` class and that is the method that is called. 

In [ ]:
cherries = MultiFruit2('cherries', 2.99, 20)
cherries.display()

In `MultiFruit3`, we inherit the constructor from Fruit, which means we can't specify the quantity when we construct a `MultiFruit3`. We add the quantity later using property assignment (`strawberries.quantity = 12`). We also override the display method and print out that we can't display a MultiFruit3 (for whatever reason).

In [ ]:
class MultiFruit3(Fruit):
    def per_item_price(self):
        return round(self.price / self.quantity, 2)
    
    def display(self):
        print('No, cannot display MultiFruit3')
strawberries = MultiFruit3('strawberry', 3.79)
strawberries.quantity = 12
strawberries.display()
print(strawberries.per_item_price())

We could also use the `super()` function in other methods of our class. For instance, if we want to calculate the "per item" discount on a `MultiFruit`, we could override the `discount()` method to do our extra calculation, before we call the method in the `Fruit` **super**class. 

In [ ]:
class MultiFruit2(Fruit):
    def __init__(self, name, price, quantity):
        print('In MultiFruit2 constructor __init__')
        self.name = name
        self.price = price
        self.quantity = quantity

    def discount(self, percent):
        if not hasattr(self, 'per_item_discount_percent'):
            self.per_item_discount_percent = percent / self.quantity
        else:
            self.per_item_discount_percent = self.per_item_discount_percent + (100 - self.per_item_discount_percent) * (1 - (percent/100/self.quantity))
        super().discount(percent)


grapes = MultiFruit2('grapes', 0.89, 10)
print(grapes.price)
grapes.discount(20)
print(grapes.price)
print(grapes.per_item_discount_percent)

Another powerful feature of inheritance is that since a `MultiFruit` "is a" `Fruit`, you can use the former anywhere the latter is required.

In [ ]:
def make_fruit_basket(fruits):
    names = [fruit.name for fruit in fruits]
    total_price = sum(fruit.price for fruit in fruits)
    return 'I made you a fruit basket with %s. It costs %.2f.' % (', '.join(names), total_price)

print(make_fruit_basket([apple, grapes]))

The fruit basket method just cares about the name and price, which all fruits (including multifruits) have. It doesn't matter that grapes also have a quantity.

## `isinstance`

Sometimes, you will get passed a fruit, and you will have different logic depending on if it is a `Fruit` or a `MultiFruit`. While not exactly encouraged, you can use an `isinstance` check to make a determination.

In [ ]:
assert isinstance(apple, Fruit)
assert not isinstance(apple, MultiFruit)
assert isinstance(grapes, Fruit)
assert isinstance(grapes, MultiFruit)

Although there are superior ways of designing your OOP programs, it's good to know that `isinstance` is there if we absolutely need it. It's far better than checking the property `.__class__` or using the built-in `type()` directly, because it inspects class hierarchies.

In [ ]:
print(type(grapes))
print(grapes.__class__)

# Exercise - Implementing a class

TBD

# Appendix - Decorators

Technically, a decorator is a function that is passed a function and returns a function. However, they look more like "annotations" in other languages, which is generally what they're used for. Decorators will be a fairly big part of our class on Web Apps with Flask, so let's discuss them now.

In our `MultiFruit` class, the per_item_price depends solely on the price and quantity of the object. We can use a decorator to mark it as a property, and then have the ability to access it without "calling" it (though the underlying function will still be called). Let's see that in action.

In [13]:
class MultiFruit3(Fruit):
    def __init__(self, name, price, quantity):
        super().__init__(name, price)
        self.quantity = quantity
    
    def on_sale(self):
        return True

    @property
    def per_item_price(self):
        return round(self.price / self.quantity, 2)
    
pears = MultiFruit3('pear', 1.65, 4)
print(pears.on_sale())
print(pears.per_item_price)

In Fruit constructor __init__
True
0.41


In [ ]:
def first_decorator(func):
  print('Replacing print_hello...')
  def fooled_you_but_still_said_it():
    func()
  return fooled_you_but_still_said_it

print_hello = @first_decorator(
def print_hello():
  print('Hello')
)

print('About to say hello...')
print_hello()

Replacing print_hello...
About to say hello...
Goodbye!!!


What's going on here? The Python interpreter saw the **decorator** called `property`(the @ sign is just how the decorator is invoked) and extended the behavior of the decorated function in some way. In this case, it created a property on the object that has the same name as the function that was decorated. There are lots of built-in Python decorators for various tasks. And we can write our own!

Let's say we wanted to print a message before and after one of our Fruit methods was called. We could copy paste print statements into each method body, but that's repetitive and error-prone. What happens when we want to change the message, but forget to change it everywhere.

Instead, we can use a decorator. Let's see how our decorator would look before we implement it.

In [ ]:
class Fruit3:
    def __init__(self, name, price):
        self.name = name
        self.price = price
    
    @log
    def discount(self, amount=.20):
        self.price = round(self.price * (1 - amount), 2)
    
    @log
    def double_default_discount(self):
        self.discount()
        self.discount()

Now we need to implement the `log` decorator. The syntax for defining a decorator is nothing special, recalling that a decorator is simply a function that is passed a function and returns a function. The function that is passed is the function that was decorated. The funcion we return is what will get called in its place.

In [ ]:
def log(func):  # It takes the original function
    def func_with_log(*args, **kwargs):
        # This is the function that will actually
        # get called. We take in *args and **kwargs so
        # we can pass them through to the original.
        print(f'About to call {func.__name__}')
        func(*args, **kwargs)
        print(f'Done calling {func.__name__}')
    return func_with_log  # Return a function

In [ ]:
f = Fruit3('banana', 0.89)
f.discount()